In [1]:
import dynet_config
dynet_config.set(autobatch=True)

In [55]:
import dynet as dy
import numpy as np
from time import time
import torch
from torch import nn

In [3]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return x_train,y_train,x_valid,y_valid

def normalize(x, m, s): return (x-m)/s

In [4]:
x_train, y_train, x_valid, y_valid = get_data()

In [76]:
class MyModel:
    def __init__(self, n_in, n_hid, n_out, param_coll=None):
        self.params = dy.ParameterCollection(parent=param_coll)
        self.w1 = self.params.add_parameters((n_hid, n_in), init='he')
        self.b1 = self.params.add_parameters(n_hid, init=0)
        self.w2 = self.params.add_parameters((n_out, n_hid), init='he')
        self.b2 = self.params.add_parameters(n_out, init=0)
        
    def __call__(self, inp):
        x = dy.inputTensor(inp)
        h = dy.rectify(self.w1*x + self.b1)
        out = self.w2*h + self.b2
        return dy.softmax(out)
        

In [77]:
m = MyModel(28**2, 128, 10)
x = x_train[0].reshape((-1))
dy.renew_cg()
m(x).npvalue()

array([0.079729, 0.092244, 0.098256, 0.102452, 0.105043, 0.09444 , 0.121233, 0.080559, 0.09715 , 0.128894])

In [60]:
trainer = dy.SimpleSGDTrainer(m.params, learning_rate=0.05)
losses = []
for i in range(N):
    x = x_train[i].reshape((-1))
    y = y_train[i]
    dy.renew_cg()
    out = m(x)
    loss = dy.pickneglogsoftmax(out, y)
    losses.append(loss.value())
    if i%100 == 0:
        print(i, np.mean(losses[-100:]))
    loss.backward()
    trainer.update()
    

0 2.432094097137451
100 1.7768647646903992
200 1.7502624487876892
300 1.7554705500602723
400 1.6869235706329346
500 1.71437633395195
600 1.7532142293453217
700 1.7591930544376373
800 1.716213254928589
900 1.7093428778648376
1000 1.6817039513587952
1100 1.7346288418769837
1200 1.6686877608299255
1300 1.695107058286667
1400 1.6704911816120147
1500 1.6093202817440033
1600 1.6439761245250701
1700 1.5946958756446838
1800 1.5623244154453277
1900 1.6146128225326537
2000 1.657301322221756
2100 1.6303002750873565
2200 1.5923677730560302
2300 1.6148867189884186
2400 1.6100420451164246
2500 1.6301289594173431
2600 1.529574282169342
2700 1.588559353351593
2800 1.61329074382782
2900 1.5836220383644104
3000 1.604748250246048
3100 1.6212855911254882
3200 1.570449630022049
3300 1.5888686001300811
3400 1.5823408710956572
3500 1.5624805617332458
3600 1.6133106231689454
3700 1.629463859796524
3800 1.5688932573795318
3900 1.5699991524219512
4000 1.571483633518219
4100 1.5864513528347015
4200 1.59620012998

34000 1.4946512770652771
34100 1.546235327720642
34200 1.513168479204178
34300 1.5129330885410308
34400 1.5485647428035736
34500 1.5827400231361388
34600 1.529432498216629
34700 1.6110585379600524
34800 1.5906141436100005
34900 1.6019719624519348
35000 1.5633963191509246
35100 1.511797341108322
35200 1.5520746552944182
35300 1.5717927134037017
35400 1.5241712260246276
35500 1.5967498338222503
35600 1.5138535845279693
35700 1.6004198622703552
35800 1.4935953509807587
35900 1.5199439013004303
36000 1.580667301416397
36100 1.549157291650772
36200 1.555746476650238
36300 1.5288743734359742
36400 1.5211778998374939
36500 1.5586398255825042
36600 1.526950445175171
36700 1.4972843146324157
36800 1.570512820482254
36900 1.5732325994968415
37000 1.5332637155056
37100 1.60986763715744
37200 1.5920192801952362
37300 1.5841328656673432
37400 1.6529848408699035
37500 1.6205834233760834
37600 1.5581015145778656
37700 1.572902010679245
37800 1.5586159944534301
37900 1.5294333493709564
38000 1.5873537

In [80]:
trainer = dy.SimpleSGDTrainer(m.params, learning_rate=0.02)
losses = []
dy.renew_cg()
N = 50000
bs = 64
for i in range(N):
    x = x_train[i].reshape((-1))
    y = y_train[i]
    out = m(x)
    loss = dy.pickneglogsoftmax(out, y)
    losses.append(loss)
    if i%bs == 0 or i+1==N:
        batch_loss = dy.esum(losses)
        print(i, batch_loss.value()/len(losses))
        batch_loss.backward()
        trainer.update()
        losses = []
        dy.renew_cg()
    

0 1.4611501693725586
64 1.5257266759872437
128 1.4916588068008423
192 1.5448004007339478
256 1.5270220041275024
320 1.5542018413543701
384 1.5001574754714966
448 1.5094587802886963
512 1.539848804473877
576 1.486899971961975
640 1.5697349309921265
704 1.5245988368988037
768 1.5388603210449219
832 1.5291959047317505
896 1.5699678659439087
960 1.5208698511123657
1024 1.5474525690078735
1088 1.5825386047363281
1152 1.630885362625122
1216 1.4944854974746704
1280 1.5760196447372437
1344 1.49660062789917
1408 1.6178611516952515
1472 1.4768686294555664
1536 1.5399712324142456
1600 1.5335816144943237
1664 1.5023202896118164
1728 1.5074198246002197
1792 1.492709994316101
1856 1.5214647054672241
1920 1.4876152276992798
1984 1.53502357006073
2048 1.512630581855774
2112 1.5080833435058594
2176 1.4939451217651367
2240 1.5347241163253784
2304 1.47165846824646
2368 1.4894624948501587
2432 1.5239888429641724
2496 1.530247449874878
2560 1.4873870611190796
2624 1.4835458993911743
2688 1.505164623260498


23168 1.5043919086456299
23232 1.5213098526000977
23296 1.4755232334136963
23360 1.4790937900543213
23424 1.508030891418457
23488 1.5035940408706665
23552 1.5084627866744995
23616 1.4926844835281372
23680 1.5118873119354248
23744 1.5689765214920044
23808 1.4905478954315186
23872 1.5779151916503906
23936 1.4974889755249023
24000 1.5122361183166504
24064 1.4939343929290771
24128 1.477524995803833
24192 1.4787678718566895
24256 1.5080413818359375
24320 1.5547895431518555
24384 1.4933503866195679
24448 1.493350625038147
24512 1.554845929145813
24576 1.5450505018234253
24640 1.5255792140960693
24704 1.5353024005889893
24768 1.5134810209274292
24832 1.5238639116287231
24896 1.5191643238067627
24960 1.5060104131698608
25024 1.5013837814331055
25088 1.4695603847503662
25152 1.479547381401062
25216 1.5499321222305298
25280 1.5066709518432617
25344 1.5420253276824951
25408 1.4790724515914917
25472 1.467074990272522
25536 1.524131417274475
25600 1.5369540452957153
25664 1.4977314472198486
25728 1

45312 1.5229051113128662
45376 1.4708529710769653
45440 1.5390983819961548
45504 1.6075910329818726
45568 1.506105899810791
45632 1.5286684036254883
45696 1.522745132446289
45760 1.477993369102478
45824 1.5545072555541992
45888 1.521725058555603
45952 1.5069857835769653
46016 1.5280741453170776
46080 1.5562856197357178
46144 1.5104949474334717
46208 1.5231152772903442
46272 1.520784854888916
46336 1.603542685508728
46400 1.5524883270263672
46464 1.5737149715423584
46528 1.476445198059082
46592 1.4800589084625244
46656 1.4792454242706299
46720 1.476559042930603
46784 1.5235002040863037
46848 1.4761781692504883
46912 1.5103992223739624
46976 1.5182164907455444
47040 1.5083500146865845
47104 1.4697827100753784
47168 1.5150303840637207
47232 1.530168056488037
47296 1.612855315208435
47360 1.5077024698257446
47424 1.5245171785354614
47488 1.5629627704620361
47552 1.490010142326355
47616 1.550097942352295
47680 1.495949625968933
47744 1.5158026218414307
47808 1.4982489347457886
47872 1.50275

In [8]:
x_train[0].shape

(784,)

In [130]:
class Dataset:
    def __init__(self, xs, ys, shuffle=True):
        assert len(xs) == len(ys)
        self.xs, self.ys, self.shuffle = xs, ys, shuffle
    def __len__(self):
        return len(self.xs)
    def __getitem__(self, i):
        return self.xs[i], self.ys[i]
    def __iter__(self):
        if self.shuffle: idxs = np.random.permutation(len(self))
        else:       idxs = list(range(len(self)))
        for i in range(len(self)):
            yield self[idxs[i]]
    def iter_batches(self, bs=64):
        if self.shuffle: idxs = np.random.permutation(len(self))
        else:       idxs = list(range(len(self)))
        batch_start = 0
        while batch_start<len(self):
            yield self[batch_start:batch_start+bs]
            batch_start += bs
            
ds = Dataset(x_train[:10], y_train[:10], False)
for d in ds:
    print(d)

(array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 5)
(array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 0)
(array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 4)
(array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 1)
(array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 9)
(array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 2)
(array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 1)
(array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 3)
(array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 1)
(array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 4)


In [107]:
def ifnone(x, default):
    return x if x is not None else default

class Model:
    def __init__(self, parent=None, name=None):
        if parent: self.params = parent.add_child(self, name)
        else:      self.params = dy.ParameterCollection()
        self.children = []
        self._training = True

    def __call__(self, x):
        raise NotImplementedError
    def save(self, fname):
        self.params.save(fname)
    def load(self, fname):
        self.params.populate(fname)
    def add_child(self, child, name):
        self.children.append(child)
        return self.params.add_subcollection(name)
    def __repr__(self):
        return '\n'.join(repr(child) for child in self.children)
    def train(self): self.training = True
    def eval(self):  self.training = False
        
class Linear(Model):
    def __init__(self, n_in, n_out, activ=dy.rectify, parent=None):
        super().__init__(parent)
        self.n_in, self.n_out, self.activ = n_in, n_out, activ
        self.w = self.params.add_parameters((n_out, n_in), init='he')
        self.b = self.params.add_parameters(n_out, init=0)
        self.activ = self.activ
        
    def __call__(self, x):
        if self.activ: return self.activ(self.w*x + self.b)
        else:          return self.w*x + self.b
    def __repr__(self):
        return f"Linear layer: in={self.n_in}, out={self.n_out}, activation={self.activ.__name__ if self.activ else 'no_op'}"
    
class Model1(Model):
    def __init__(self, n_in, n_hid, n_out, parent=None):
        super().__init__(parent)
        self.lin1 = Linear(n_in, n_hid, parent=self)
        self.lin2 = Linear(n_hid, n_out, activ=None, parent=self)
        
    def __call__(self, x):
        x = dy.inputTensor(x)
        return self.lin2(self.lin1(x))
        

In [146]:
m = Model1(28**2, 128,  10)
np.argmax(m(train_ds[0][0]).npvalue())

2

In [106]:
x, y = next(iter(train_ds))
dy.renew_cg()
out = m(x)
out.forward()
loss = loss_func(out, y)
loss.forward()
t0 = time()
?dy.forward
print((time() - t0)*10**6)
%timeit -n 1 out.npvalue()

798.7022399902344
The slowest run took 12.99 times longer than the fastest. This could mean that an intermediate result is being cached.
21 µs ± 21.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [151]:
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid, shuffle=False)

In [157]:
def evaluate(model, ds, loss_func, bs=64):
    loss_sum, n_cor = 0, 0
    for bx, by in ds.iter_batches(bs):
        losses, logits = [], []
        dy.renew_cg()
        for x, y in zip(bx, by):
            out = m(x)
            losses.append(loss_func(out, y))
            logits.append(out)
        dy.forward(logits)
        loss_sum += dy.esum(losses).value()
        n_cor += sum(np.argmax(log.npvalue()) == y for log, y in zip(logits, by))
    return loss_sum/len(ds), n_cor/len(ds)

In [180]:
m = Model1(28**2, 128,  10)
n_epochs = 1
bs = 512
trainer = dy.SimpleSGDTrainer(m.params, learning_rate=0.5)
loss_func = dy.pickneglogsoftmax
def fit():
    losses = []
    dy.renew_cg()
    for epoch in range(n_epochs):
        for i, (x, y) in enumerate(train_ds):
            losses.append(loss_func(m(x), y))
            if i>0 and i%bs==0 or i+1==len(train_ds):
                loss = dy.esum(losses)
                loss.backward()
                if i%100==0 : print(i, loss.value()/len(losses))
                trainer.update()
                losses = []
                dy.renew_cg()
        avg_loss, acc = evaluate(m, valid_ds, loss_func)
        print(epoch, f' loss: {avg_loss}, accuracy: {acc}')
                
fit() 

12800 1.502548098564148
25600 1.5826669931411743
38400 1.0932579040527344
0  loss: 1.2846816074371339, accuracy: 0.6591


In [174]:
?dy.ParameterCollection

In [123]:
bx, by = train_ds[:10]
list(zip(bx, by))

[(array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 5),
 (array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 0),
 (array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 4),
 (array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 1),
 (array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 9),
 (array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 2),
 (array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 1),
 (array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 3),
 (array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 1),
 (array([0., 0., 0., 0., ..., 0., 0., 0., 0.], dtype=float32), 4)]

In [173]:
trainer.learning_rate = 0.1
?trainer.restart